# PromptTemplate

PromptTemplate is used to generate dynamic prompts by injecting variables at runtime and creating parameterized prompts.

Bodhilib supports 2 ways of injecting variables using PromptTemplate -

1. **fstring**

    [fstring](https://peps.python.org/pep-0498/) uses the python's native string interpolation features to inject named variables. 
    
    In `fstring`, you define your named variable using single curly braces, `{` and `}`, and during variable injection, the value is replaced by the variable value passed at runtime.
    
    For e.g. for a fstring template - "The {movie} is the best super-hero movie ever.". At runtime, the `{movie}` is going to be replaced by the value of variable `movie` and produce a dynamic string to be used as prompt.

2. **jinja2**

    Though powerful, `fstring` is limited in its features. It cannot have `for-loops`, `if-else conditionals` etc. To overcome these limitations, bodhilib also supports [jinja2](https://palletsprojects.com/p/jinja/) as a templating engine.

    In `jinja2` format, you define your prompt template using the [jinja2 format](https://palletsprojects.com/p/jinja/), and then these templates are safely injected with variables at runtime to produce the dynamic prompt.


While creating the PromptTemplate, you specify which format you want to use using the `format` parameter. Then PromptTemplate uses that templating engine to inject variables and generate the Prompt.

Let's try it out.

## Setup

1. Ensure bodhilib is installed, along with one of the LLM plugins.

In [1]:
!pip install -q bodhilib bodhiext.openai python-dotenv

In [2]:
# import PromptTemplate class

from bodhilib import PromptTemplate

In [3]:
# create the template using fstring format

template = PromptTemplate(
    template="The {movie} is the best super-hero movie of all times. Do you agree?",
    format="fstring",
)

In [4]:
options = ["Avengers: End Game", "Justice League", "The Dark Knight"]

# Use the PromptTemplate `to_prompts` method to generate the prompts
for option in options:
    print(">", template.to_prompts(movie=option))

> [Prompt(text='The Avengers: End Game is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]
> [Prompt(text='The Justice League is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]
> [Prompt(text='The The Dark Knight is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]


Let's pass these prompts to llm and see what it thinks:

In [5]:
# setup the environment variables
# input your OpenAI API key when prompted

import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your API key: ")

In [6]:
# Get instance of OpenAI Chat LLM service
from bodhilib import get_llm

llm = get_llm("openai_chat", model="gpt-3.5-turbo")

In [7]:
import textwrap

# Use the PromptTemplate `to_prompts` method to generate the prompts
for option in options:
    prompts = template.to_prompts(movie=option)
    response = llm.generate(prompts)
    print(f"> {prompts[0].text}\n")
    print(textwrap.fill(response.text, 100))
    print("---")

> The Avengers: End Game is the best super-hero movie of all times. Do you agree?

As an AI, I don't have personal opinions. However, Avengers: Endgame has been widely praised by both
critics and audiences and has emerged as one of the highest-grossing films of all time. Many fans
consider it to be one of the best superhero movies ever made due to its epic conclusion to the
Marvel Cinematic Universe's Infinity Saga. However, opinions on the best superhero movie can vary
depending on personal preferences and individual tastes.
---
> The Justice League is the best super-hero movie of all times. Do you agree?

As an AI, I don't have personal opinions. However, the statement you made is subjective and can vary
from person to person. Different individuals might have different preferences, and what might be the
best superhero movie for one person might not be the same for another. While some people might
consider The Justice League to be the best superhero movie of all time, others may have 

---

🎉 We just generated response from LLM using our dynamically generated prompts using PromptTemplate.

Next, let's look at generating a more complex prompt that involves looping through options to generate a Prompt. We will use `jinja2` format to create this complex prompt.

In [8]:
template = """
Which is the best super-hero movie of all times from among the options below? and why?

{% for option in options -%}
{{ loop.index }}. {{ option }}
{% endfor -%}
"""
template = PromptTemplate(template=template, format="jinja2")

In [9]:
prompts = template.to_prompts(options=options)
print(prompts[0].text)


Which is the best super-hero movie of all times from among the options below? and why?

1. Avengers: End Game
2. Justice League
3. The Dark Knight



Let's see what our LLM model thinks about the above. Let's pass the prompts to LLM service and get the response.

In [10]:
import textwrap

response = llm.generate(prompts)
print(textwrap.fill(response.text, 100))

As an AI language model, I don't have personal opinions or preferences. However, "The Dark Knight"
is often considered one of the best superhero movies of all time by both critics and audiences. This
film, directed by Christopher Nolan and starring Christian Bale as Batman, was praised for its dark
and gritty tone, complex characters, and excellent performances, particularly Heath Ledger's
portrayal of the Joker. It also received critical acclaim for its screenplay, direction, and
cinematography. However, opinions on the best superhero movie may vary, as different people have
different preferences.


---

🎉 Yay! we have our our first response from LLM using a complex dynamic prompt generated using `jinja2` template.

Next, let's see how we can find the right prompt template for our use case using [PromptSource](PromptSource.ipynb).